In [ ]:
!pip install -U sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

d:\Users\vramo\anaconda3\envs\deep-cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Users\vramo\anaconda3\envs\deep-cuda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Users\vramo\anaconda3\envs\deep-cuda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:

def get_embedding(docs):
    texts = []
    urls = []
    titles = []
    for doc in docs:
        text = doc['title'] + '\n' + doc['body']
        texts.append(text)
        urls.append(doc['url'])
        titles.append(doc['title'])
    embeddings = model.encode(texts)
    return embeddings, urls, titles



In [6]:
from glob import glob

# load the file names with glob('..\\data\\newsapiai\\*.json') and omit is _processed in the name

files = glob('..\\data\\newsapiai\\*.json')
files = [file for file in files if '_processed' not in file]
files


['..\\data\\newsapiai\\newsai_2024-05-05_06-55-26.json']

In [4]:
import json
import os
all_embeddings = []
all_urls = []
all_titles = []
for file in files:
    docs = []
    with open(file, 'r') as f:
        for line in f:
            docs.append(json.loads(line))
    print(len(docs))
    print(docs[0])
    print(type(docs[0]))
    embeddings, urls, titles = get_embedding(docs)
    all_embeddings.extend(embeddings)
    all_urls.extend(urls)
    all_titles.extend(titles)
    # rename file adding _processed to the name
    os.rename(file, file.replace('.json', '_processed.json'))
    
    

    
    

375
{'uri': '8101960666', 'lang': 'eng', 'isDuplicate': False, 'date': '2024-04-28', 'time': '23:59:52', 'dateTime': '2024-04-28T23:59:52Z', 'dateTimePub': '2024-04-28T23:58:03Z', 'dataType': 'news', 'sim': 0, 'url': 'https://www.deccanherald.com/elections/india/in-bellary-political-fortunes-at-stake-as-bjp-s-sriramulu-takes-on-cong-s-tukaram-2998765', 'title': "In Bellary, political fortunes at stake as BJP's Sriramulu takes on Congress' Tukaram", 'body': 'Bharatiya Janata Party Bellary Lok Sabha candidate B Sriramulu is hopeful that "he who resurrected Sri Rama in Ayodhya will ensure his political rebirth in this election." The defeat in the Bellary Rural Assembly constituency in 2023 seems to have devastated him. In every public rally, Sriramulu has been emotionally appealing to voters to ensure he is not pushed to political oblivion.\n\nOn the contrary, Congress candidate and four-time MLA from Sandur E Tukaram is in two minds: whether to fight or not to fight wholeheartedly. Party

In [19]:
all_titles[:5], all_urls[:5], all_embeddings[:5]

(["In Bellary, political fortunes at stake as BJP's Sriramulu takes on Congress' Tukaram",
  'Tapping TABOR: Colorado lawmakers turn to surplus dollars to fund tax credits',
  "Lance Bass Performs *NSYNC's 'Bye Bye Bye' at Egyptian Wedding of Billionaire CEO and Ex-WWE Star",
  'Nobody will dare to change the Constitution knowing its political ramifications: Kamble',
  'Vote For Bjp & Allies In Assam: Maulanas'],
 ['https://www.deccanherald.com/elections/india/in-bellary-political-fortunes-at-stake-as-bjp-s-sriramulu-takes-on-cong-s-tukaram-2998765',
  'https://gazette.com/news/colorado-legislature/colorado-tabor-surplus-tax-credits/article_14ffa4f4-be47-5684-a11b-f47ecb22975c.html',
  'https://people.com/lance-bass-sings-nsync-bye-bye-bye-at-egyptian-wedding-billionaire-ceo-wwe-star-video-8640190',
  'https://indianexpress.com/article/cities/mumbai/nobody-will-dare-to-change-the-constitution-knowing-its-political-ramifications-kamble-9296108/',
  'https://timesofindia.indiatimes.com/c

In [11]:


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_similar(query, embeddings, urls, titles, top_k=5):
    query = model.encode([query])
    cos_sim = cosine_similarity(query, embeddings)
    cos_sim = cos_sim[0]
    indices = np.argsort(cos_sim)[-top_k:][::-1]
    return [(titles[i], urls[i]) for i in indices]

get_similar('Donald Trump robs money', all_embeddings, all_urls, all_titles)

[("That scowl. The gag order. Frightened jurors. Who's on trial, a former president or a mob boss?",
  'https://www.democraticunderground.com/1016376719'),
 ('DeSantis makes nice with Trump in Miami: reports',
  'https://www.businessinsider.com/desantis-trump-presidential-campaign-fundraising-meeting-florida-musk-reports-2024-4'),
 ('DeSantis makes nice with Trump in Miami: reports',
  'https://www.businessinsider.com/desantis-trump-presidential-campaign-fundraising-meeting-florida-musk-reports-2024-4'),
 ('DeSantis, Donald Trump Discuss Détente In Miami',
  'https://dailycaller.com/2024/04/28/ron-desantis-donald-trump-detente-miami-florida-washington-post/'),
 ('US takes big step toward making Russia pay for Ukraine invasion',
  'https://www.atlanticcouncil.org/blogs/ukrainealert/us-takes-big-step-toward-making-russia-pay-for-ukraine-invasion/')]